## Summary

Calculate features using [ProteinSolver](https://gitlab.com/ostrokach/proteinsolver).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_proteinsolver.ipynb)"

export DATASET_NAME="cagi6-sherloc"
export DATASET_PATH="30_cagi6_sherloc/input-data-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=4182

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-3500 --gres=gpu:t4:1 ../scripts/run_notebook_gpu.sh

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=181-200 --gres=gpu:v100:1 ../scripts/run_notebook_gpu.sh

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=201-306 ../scripts/run_notebook_cpu.sh
```

---

## Imports

In [1]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import torch
from elaspic2.plugins.proteinsolver import (
    ProteinSolver,
    ProteinSolverAnalyzeError,
    ProteinSolverBuildError,
)
from kmbio import PDB
from tqdm.notebook import tqdm

## Parameters

In [2]:
NOTEBOOK_DIR = Path("31_run_proteinsolver").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_proteinsolver')

In [3]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir
    
print(tempfile.gettempdir())

/tmp


In [4]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, TASK_ID, TASK_COUNT

(None, None, None)

In [5]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "cagi6-sherloc"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
    )
    TASK_ID = 3500
    TASK_COUNT = 4182
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    assert TASK_ID is not None
    assert TASK_COUNT is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('cagi6-sherloc',
 '/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_cagi6_sherloc/input-data-gby-protein.parquet',
 3500,
 4182)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

## Workspace

### Initialize model

In [7]:
ProteinSolver.load_model(device=device)

### Load data

In [8]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

4182

In [9]:
assert TASK_COUNT == pfile.num_row_groups

In [10]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [11]:
display(input_df.head(2))
print(len(input_df))

,protein_id,mutation_id,mutation,effect,sequence,structure,alignment
0,Q14191,"[NM_000553.4:c.487G>C, NM_000553.4:c.487G>T, N...","[D163H, D163Y, D163V, S1013T, N166S, L169Q, C1...","[Uncertain significance, Uncertain significanc...",MSEKKLETTAQQRKCPEWMNVQNKRCAVEERKACVRKSVFEDDLPF...,HEADER ...,"[>101\n, MSEKKLETTAQQRKCPEWMNVQNKRCAVEERKACVRK..."


1


In [12]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

['mutation_id', 'mutation', 'effect']

In [13]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [14]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [15]:
def worker(mutation, data):
    try:
        results = ProteinSolver.analyze_mutation(f"A_{mutation}", data)
    except ProtBertAnalyzeError as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"proteinsolver_core_{key}": value for key, value in results.items()}
        return results

In [16]:
results = []
for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
        with open(tmp_file.name, "wt") as fout:
            fout.write(tup.structure)
        data = ProteinSolver.build(tmp_file.name, tup.sequence, None, remove_hetatms=False)

    for mutation_idx, mutation in enumerate(tqdm(tup.mutation, leave=False)):
        if not validate_mutation(mutation):
            continue

        result = worker(mutation, data)
        if result is None:
            continue

        results.append(
            {
                "protein_id": tup.protein_id,
                "mutation": mutation,
            }
            | {field: getattr(tup, field)[mutation_idx] for field in iterable_fields}
            | result
        )

results_df = pd.DataFrame(results)

In [17]:
display(results_df.head(2))
print(len(results_df))

,protein_id,mutation,mutation_id,effect,proteinsolver_core_score_wt,proteinsolver_core_score_mut,proteinsolver_core_features_residue_wt,proteinsolver_core_features_protein_wt,proteinsolver_core_features_residue_mut,proteinsolver_core_features_protein_mut
0,Q14191,D163H,NM_000553.4:c.487G>C,Uncertain significance,0.090927,0.016798,"[0.783461332321167, 0.28929370641708374, -0.55...","[-1.5107340812683105, 0.8064138293266296, 0.05...","[-3.101989507675171, 0.1873071789741516, -0.24...","[-1.5133936405181885, 0.8066174387931824, 0.05..."
1,Q14191,D163Y,NM_000553.4:c.487G>T,Uncertain significance,0.090927,0.007515,"[0.7834614515304565, 0.2892940640449524, -0.55...","[-1.5107340812683105, 0.8064139485359192, 0.05...","[3.1103622913360596, 0.9690061211585999, -0.14...","[-1.5084962844848633, 0.8081052899360657, 0.05..."


3


### Save results

In [18]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_proteinsolver/cagi6-sherloc/shard-3500-of-4182.parquet')

In [19]:
if not DEBUG:
    pq.write_table(pa.Table.from_pandas(results_df, preserve_index=False), output_file)